In [1]:
import sys

import polars as pl
import torch


'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '/Users/igwanhyeong/PycharmProjects/data_research/raw_data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/raw_data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR

df = pl.read_parquet(DIR + 'target_dyn_demand_weekly.parquet')

True
1
12.8
2.9.0.dev20250716+cu128
NVIDIA GeForce RTX 5080
2.9.0.dev20250716+cu128


In [2]:
# 타입 정리
df = df.with_columns(
    pl.col("oper_part_no").cast(pl.Utf8),
    pl.col("demand_dt").cast(pl.Int64, strict=False),
    pl.col("demand_qty").cast(pl.Float64),
).sort(["oper_part_no", "demand_dt"])

In [3]:
import numpy as np

# -----------------------------
# 파트별 랜덤 파라미터(불량률, 보증개월) 생성
# -----------------------------
rng = np.random.default_rng(seed=42)

parts = df.select(pl.col("oper_part_no").unique().sort()).to_series().to_list()
n = len(parts)

In [4]:
# 불량률: U(0.01, 0.05)
defect_rates = rng.uniform(0.01, 0.05, size=n)

In [5]:
# Warranty months: {6, 12, 24, 36} 중 균등 샘플
wty_choices = np.array([6, 12, 24, 36], dtype=int)
wty_months = rng.choice(wty_choices, size=n, replace=True)

In [6]:
# 파라미터 테이블(파트 단위)
param_df = pl.DataFrame({
    "oper_part_no": parts,
    "defect_rate": defect_rates,
    "warranty_months": wty_months,
})

### Sample Data 만들기

부품별 주차 그리드 생성 및 demand_qty 결측 0 채움

warranty_months → warranty_weeks(=×4) 계산

(선택) 원리 일관성 유지: 수리수요를 판매에 귀속시키는 기원 주차(origin) 계산 → sales_base = demand_qty/defect_rate 를 origin_yyyyww에 더함

부품별로 판매 시작/종료 구간(약 2년=104주) 을 결정적 난수(해시) 로 생성

그 구간에서만 sales_qty가 발생하도록 마스킹

판매 시작일부터 종료일까지 sales_cum(누적), 종료 이후에는 첫 주부터 하나씩 빼며 감소하는 sales_cum_decay 생성

보증구간 플래그 in_warranty: first_sale_date(=판매 시작)부터 warranty_weeks까지 1, 이후 0

In [7]:
import polars as pl
from datetime import date, timedelta

# ======================================================
# 안전 유틸 (예외 없음) — 잘못된 입력은 None 반환
# ======================================================
def safe_yyyyww_to_monday(yyyyww):
    if yyyyww is None:
        return None
    try:
        y = int(yyyyww) // 100
        w = int(yyyyww) % 100
        if y < 1900 or y > 9999 or w < 1 or w > 53:
            return None
        return date.fromisocalendar(y, w, 1)
    except Exception:
        return None

def safe_date_to_yyyyww(d):
    if d is None:
        return None
    try:
        y, w, _ = d.isocalendar()
        if y < 1900 or y > 9999 or w < 1 or w > 53:
            return None
        return int(y * 100 + w)
    except Exception:
        return None

def safe_add_weeks_to_yyyyww(yyyyww, delta_weeks):
    if yyyyww is None or delta_weeks is None:
        return None
    try:
        d0 = safe_yyyyww_to_monday(yyyyww)
        if d0 is None:
            return None
        return safe_date_to_yyyyww(d0 + timedelta(weeks=int(delta_weeks)))
    except Exception:
        return None

# ======================================================
# 0) 입력 전처리
#  - df:         oper_part_no | demand_dt(yyyyww) | demand_qty
#  - param_df:   oper_part_no | defect_rate | warranty_months
# ======================================================
demand_df = (
    df.rename({"demand_dt": "yyyyww"})
      .with_columns([
          pl.col("oper_part_no").cast(pl.Utf8),
          pl.col("yyyyww").cast(pl.Int64, strict=False),   # 넉넉히 Int64
          pl.col("demand_qty").cast(pl.Float64),
      ])
      .group_by(["oper_part_no", "yyyyww"])
      .agg(pl.col("demand_qty").sum())
)

# 파라미터 안전화(결측/비정상값 방어)
param_df_std = (
    param_df.with_columns([
        pl.col("oper_part_no").cast(pl.Utf8),
        (pl.col("warranty_months") * 4).cast(pl.Int64).alias("warranty_weeks_raw"),
        pl.col("defect_rate").cast(pl.Float64).alias("defect_rate_raw"),
    ])
    .with_columns([
        pl.col("warranty_weeks_raw").fill_null(0).clip(lower_bound=0, upper_bound=520).alias("warranty_weeks"),
        pl.col("defect_rate_raw").fill_null(0.02).clip(lower_bound=1e-6, upper_bound=0.5).alias("defect_rate"),
    ])
    .select("oper_part_no", "defect_rate", "warranty_weeks")
)

# ======================================================
# 1) 부품별 주차 그리드(관측 min~max 사이)
# ======================================================
obs_weeks = (
    demand_df
    .with_columns(
        pl.col("yyyyww").map_elements(safe_yyyyww_to_monday, return_dtype=pl.Date).alias("week_date")
    )
    .drop_nulls(subset=["week_date"])   # 변환 실패 yyyyww 제거(정책 변경 가능)
    .select("oper_part_no", "yyyyww", "week_date")
    .unique()
)

part_bounds = (
    obs_weeks.group_by("oper_part_no")
             .agg([
                 pl.min("week_date").alias("min_date"),
                 pl.max("week_date").alias("max_date"),
             ])
)

full_grid = (
    part_bounds
    .with_columns(
        pl.date_ranges(pl.col("min_date"), pl.col("max_date"), interval="1w", closed="both").alias("grid_dates")
    )
    .explode("grid_dates")
    .rename({"grid_dates": "week_date"})
    .with_columns(
        pl.col("week_date").map_elements(safe_date_to_yyyyww, return_dtype=pl.Int64).alias("yyyyww")
    )
    .drop_nulls(subset=["yyyyww"])
    .with_columns(pl.col("yyyyww").cast(pl.Int64))
    .select("oper_part_no", "yyyyww", "week_date")
    .sort(["oper_part_no", "yyyyww"])
)

# demand 누락 0 채움
demand_filled = (
    full_grid.join(demand_df, on=["oper_part_no", "yyyyww"], how="left")
             .with_columns(pl.col("demand_qty").fill_null(0.0))
)

# ======================================================
# 2) origin 매핑을 통한 판매량의 "기저" 추정
#    - demand는 보증기간만큼 뒤의 판매에서 발생했다고 보고,
#      판매 귀속 주차(origin_yyyyww)에 demand/defect_rate를 누적
# ======================================================
dfx = (
    demand_filled.join(param_df_std, on="oper_part_no", how="left")
    .with_columns([
        pl.struct(["yyyyww", "warranty_weeks"]).map_elements(
            lambda s: safe_add_weeks_to_yyyyww(
                s["yyyyww"],
                -int(s["warranty_weeks"]) if s["warranty_weeks"] is not None else None
            ),
            return_dtype=pl.Int64
        ).alias("origin_yyyyww"),
        (pl.col("demand_qty") / pl.col("defect_rate")).alias("origin_sales_contrib"),
    ])
    .filter(pl.col("origin_yyyyww").is_not_null())
)

sales_base = (
    dfx.group_by(["oper_part_no", "origin_yyyyww"])
       .agg(pl.col("origin_sales_contrib").sum().alias("sales_qty"))
       .rename({"origin_yyyyww": "yyyyww"})
)

# full_grid에 sales_base를 붙여 결측 0
sales_full = (
    full_grid
    .join(sales_base, on=["oper_part_no", "yyyyww"], how="left")
    .join(
        demand_filled.select("oper_part_no", "yyyyww", "demand_qty"),
        on=["oper_part_no", "yyyyww"], how="left"
    )
    .with_columns([
        pl.col("sales_qty").fill_null(0.0),
        pl.col("demand_qty").fill_null(0.0),   # 안전
    ])
)

# ======================================================
# 3) 부품별 판매 시작/종료 구간(≈2년=104주) 생성 (결정적 난수 기반)
# ======================================================
WINDOW_WEEKS = 104  # ≈ 2년


bounds2 = (
    full_grid
    .group_by("oper_part_no")
    .agg([
        pl.min("week_date").alias("min_date"),
        pl.max("week_date").alias("max_date"),
        pl.count().alias("n_weeks"),
    ])
    # 1단계: span, max_offset, u01, start_offset_weeks
    .with_columns([
        (pl.col("n_weeks") - 1).alias("span_weeks"),
        (pl.col("n_weeks") - 1 - WINDOW_WEEKS).clip(lower_bound=0).alias("max_offset"),
        (pl.col("oper_part_no").hash().cast(pl.UInt64) / (2**64 - 1)).alias("u01"),
    ])
    .with_columns([
        ((pl.col("u01") * (pl.col("max_offset") + 1)).floor().cast(pl.Int64)).alias("start_offset_weeks"),
    ])
    # 2단계: 랜덤 시작/종료 계산
    .with_columns([
        (pl.col("min_date") + pl.duration(weeks=pl.col("start_offset_weeks"))).alias("start_rand"),
        (pl.col("min_date") + pl.duration(weeks=pl.col("start_offset_weeks") + WINDOW_WEEKS)).alias("end_rand_raw"),
    ])
    .with_columns([
        pl.min_horizontal([pl.col("end_rand_raw"), pl.col("max_date")]).alias("end_rand"),
    ])
    # 3단계: span < WINDOW 인 경우 오른쪽 정렬 시작점 계산
    .with_columns([
        (pl.col("max_date") - pl.duration(weeks=WINDOW_WEEKS - 1)).alias("start_right_raw"),
    ])
    .with_columns(
        pl.max_horizontal([pl.col("start_right_raw"), pl.col("min_date")]).alias("start_right")
    )
    # 4단계: 최종 sales_start_date / sales_end_date 선택
    .with_columns([
        pl.when(pl.col("span_weeks") >= WINDOW_WEEKS)
          .then(pl.col("start_rand"))
          .otherwise(pl.col("start_right"))
          .alias("sales_start_date"),
        pl.when(pl.col("span_weeks") >= WINDOW_WEEKS)
          .then(pl.col("end_rand"))
          .otherwise(pl.col("max_date"))
          .alias("sales_end_date"),
    ])
    .select("oper_part_no", "sales_start_date", "sales_end_date")
)

# 판매 구간 플래그 및 마스킹
sales_masked = (
    sales_full.join(bounds2, on="oper_part_no", how="left")
              .with_columns(
                  (
                      (pl.col("week_date") >= pl.col("sales_start_date")) &
                      (pl.col("week_date") <= pl.col("sales_end_date"))
                  ).alias("in_sales_window")
              )
              .with_columns(
                  pl.when(pl.col("in_sales_window")).then(pl.col("sales_qty")).otherwise(0.0)
                    .alias("sales_qty_active")
              )
)



C:\Users\USER\AppData\Local\Temp\ipykernel_20392\3757992689.py:166: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("n_weeks"),


In [8]:
sales_masked

oper_part_no,yyyyww,week_date,sales_qty,demand_qty,sales_start_date,sales_end_date,in_sales_window,sales_qty_active
str,i64,date,f64,f64,date,date,bool,f64
"""0001-1001""",201811,2018-03-12,0.0,5.0,2020-12-07,2022-12-05,false,0.0
"""0001-1001""",201812,2018-03-19,0.0,0.0,2020-12-07,2022-12-05,false,0.0
"""0001-1001""",201813,2018-03-26,0.0,0.0,2020-12-07,2022-12-05,false,0.0
"""0001-1001""",201814,2018-04-02,0.0,0.0,2020-12-07,2022-12-05,false,0.0
"""0001-1001""",201815,2018-04-09,0.0,0.0,2020-12-07,2022-12-05,false,0.0
…,…,…,…,…,…,…,…,…
"""ZZ90239""",202622,2026-05-25,0.0,0.0,2021-07-12,2023-07-10,false,0.0
"""ZZ90239""",202623,2026-06-01,0.0,0.0,2021-07-12,2023-07-10,false,0.0
"""ZZ90239""",202624,2026-06-08,0.0,0.0,2021-07-12,2023-07-10,false,0.0


In [9]:
from modeling_module import DateUtil

# ======================================================
# 4) 누적(sales_cum) & 종료 이후 감소 누적(sales_cum_decay)
#    - sales_cum: 판매창 내는 누적합, 종료 이후엔 고정
#    - sales_cum_decay: 종료 이후 t에서 판매 첫 주부터 1개씩 빼며 감소
#      (= total_end - prefix_first_k), 하한 0
# ======================================================
tmp = (
    sales_masked
    .with_columns(pl.arange(0, pl.len()).over("oper_part_no").alias("idx"))
)

# 판매창 인덱스 범위
idx_bounds = (
    tmp.filter(pl.col("in_sales_window"))
       .group_by("oper_part_no")
       .agg([
           pl.min("idx").alias("idx_start"),
           pl.max("idx").alias("idx_end"),
       ])
)
tmp = tmp.join(idx_bounds, on="oper_part_no", how="left")

# 누적합(cs) 및 종료총합(total_end) — 표준 cumsum()
tmp = tmp.with_columns(
    pl.col("sales_qty_active").cum_sum().over("oper_part_no").alias("cs")
)
total_end_tbl = tmp.group_by("oper_part_no").agg(pl.max("cs").alias("total_end"))
tmp = tmp.join(total_end_tbl, on="oper_part_no", how="left")

# list() 네임스페이스 없이 self-join으로 prefix(cs_at_idx) 구하기
tmp = tmp.with_columns([
    pl.coalesce([pl.col("idx_start"), pl.col("idx")]).alias("idx_start_filled"),
    pl.coalesce([pl.col("idx_end"),   pl.col("idx")]).alias("idx_end_filled"),
])

tmp = tmp.with_columns(
    pl.when(pl.col("idx_end").is_null())
      .then(0)
      .otherwise((pl.col("idx") - pl.col("idx_end_filled")).clip(lower_bound=0))
      .alias("k_after_end")
)

tmp = tmp.with_columns([
    (pl.col("idx_start_filled") + pl.col("k_after_end") - 1).alias("target_idx_raw")
]).with_columns([
    pl.min_horizontal([pl.col("target_idx_raw"), pl.col("idx_end_filled")]).alias("target_idx_capped")
])

idx_cs = tmp.select(["oper_part_no", "idx", "cs"]).rename({"cs": "cs_at_idx"})
tmp = tmp.join(
    idx_cs,
    left_on=["oper_part_no", "target_idx_capped"],
    right_on=["oper_part_no", "idx"],
    how="left",
)

tmp = tmp.with_columns(
    pl.when(pl.col("k_after_end") <= 0)
      .then(0.0)
      .otherwise(pl.col("cs_at_idx").fill_null(0.0))
      .alias("prefix_first_k")
)

tmp = tmp.with_columns([
    pl.col("cs").alias("sales_cum"),
    pl.when( (pl.col("idx") <= pl.col("idx_end_filled")) | (pl.col("idx_end").is_null()) )
      .then(pl.col("cs"))
      .otherwise( (pl.col("total_end") - pl.col("prefix_first_k")).clip(lower_bound=0.0) )
      .alias("sales_cum_decay")
])

# ======================================================
# 5) 보증 플래그 in_warranty (판매 시작일 기준)
# ======================================================
tmp = (
    tmp.join(param_df_std.select("oper_part_no", "warranty_weeks"), on="oper_part_no", how="left")
       .with_columns(
           (pl.col("sales_start_date") + pl.duration(weeks=pl.col("warranty_weeks"))).alias("warranty_end_date")
       )
       .with_columns(
           (
             (pl.col("week_date") >= pl.col("sales_start_date")) &
             (pl.col("week_date") <= pl.col("warranty_end_date"))
           ).cast(pl.Int8).alias("in_warranty")
       )
)
tmp

# result 가 최종 산출물

oper_part_no,yyyyww,week_date,sales_qty,demand_qty,sales_start_date,sales_end_date,in_sales_window,sales_qty_active,idx,idx_start,idx_end,cs,total_end,idx_start_filled,idx_end_filled,k_after_end,target_idx_raw,target_idx_capped,cs_at_idx,prefix_first_k,sales_cum,sales_cum_decay,warranty_weeks,warranty_end_date,in_warranty
str,i64,date,f64,f64,date,date,bool,f64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,date,i8
"""0001-1001""",201811,2018-03-12,0.0,5.0,2020-12-07,2022-12-05,false,0.0,0,143,247,0.0,97.660442,143,247,0,142,142,0.0,0.0,0.0,0.0,48,2021-11-08,0
"""0001-1001""",201812,2018-03-19,0.0,0.0,2020-12-07,2022-12-05,false,0.0,1,143,247,0.0,97.660442,143,247,0,142,142,0.0,0.0,0.0,0.0,48,2021-11-08,0
"""0001-1001""",201813,2018-03-26,0.0,0.0,2020-12-07,2022-12-05,false,0.0,2,143,247,0.0,97.660442,143,247,0,142,142,0.0,0.0,0.0,0.0,48,2021-11-08,0
"""0001-1001""",201814,2018-04-02,0.0,0.0,2020-12-07,2022-12-05,false,0.0,3,143,247,0.0,97.660442,143,247,0,142,142,0.0,0.0,0.0,0.0,48,2021-11-08,0
"""0001-1001""",201815,2018-04-09,0.0,0.0,2020-12-07,2022-12-05,false,0.0,4,143,247,0.0,97.660442,143,247,0,142,142,0.0,0.0,0.0,0.0,48,2021-11-08,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZZ90239""",202622,2026-05-25,0.0,0.0,2021-07-12,2023-07-10,false,0.0,313,59,163,50.208904,50.208904,59,163,150,208,163,50.208904,50.208904,50.208904,0.0,144,2024-04-15,0
"""ZZ90239""",202623,2026-06-01,0.0,0.0,2021-07-12,2023-07-10,false,0.0,314,59,163,50.208904,50.208904,59,163,151,209,163,50.208904,50.208904,50.208904,0.0,144,2024-04-15,0
"""ZZ90239""",202624,2026-06-08,0.0,0.0,2021-07-12,2023-07-10,false,0.0,315,59,163,50.208904,50.208904,59,163,152,210,163,50.208904,50.208904,50.208904,0.0,144,2024-04-15,0


In [10]:

# ======================================================
# 최종 정리 (yyyyww로 재변환; 안전 변환 사용)
# ======================================================
result = (
    tmp.with_columns([
            pl.col("week_date").map_elements(DateUtil.date_to_yyyyww, return_dtype=pl.Int64).alias("yyyyww"),
            pl.col("sales_start_date").map_elements(DateUtil.date_to_yyyyww, return_dtype=pl.Int64).alias("sales_start_yyyyww"),
            pl.col("sales_end_date").map_elements(DateUtil.date_to_yyyyww, return_dtype=pl.Int64).alias("sales_end_yyyyww"),
        ])
       .drop_nulls(subset=["yyyyww"])     # 안전: 변환 실패행 제거(정책 변경 가능)
       .with_columns([
           pl.col("yyyyww").cast(pl.Int32),
           pl.col("sales_start_yyyyww").cast(pl.Int32),
           pl.col("sales_end_yyyyww").cast(pl.Int32),
       ])
       .select(
           "oper_part_no", "yyyyww",
           "demand_qty",
           "sales_qty",            # origin 기반 기저 판매량(마스킹 전)
           "sales_qty_active",     # 판매구간 외 0 마스킹된 판매량
           "sales_start_yyyyww", "sales_end_yyyyww",
           "in_warranty",
           "sales_cum", "sales_cum_decay"
       )
       .sort(["oper_part_no", "yyyyww"])
)

oper_part_no,yyyyww,demand_qty,sales_qty,sales_qty_active,sales_start_yyyyww,sales_end_yyyyww,in_warranty,sales_cum,sales_cum_decay
str,i32,f64,f64,f64,i32,i32,i8,f64,f64
"""0001-1001""",201811,5.0,0.0,0.0,202050,202249,0,0.0,0.0
"""0001-1001""",201812,0.0,0.0,0.0,202050,202249,0,0.0,0.0
"""0001-1001""",201813,0.0,0.0,0.0,202050,202249,0,0.0,0.0
"""0001-1001""",201814,0.0,0.0,0.0,202050,202249,0,0.0,0.0
"""0001-1001""",201815,0.0,0.0,0.0,202050,202249,0,0.0,0.0
…,…,…,…,…,…,…,…,…,…
"""ZZ90239""",202622,0.0,0.0,0.0,202128,202328,0,50.208904,0.0
"""ZZ90239""",202623,0.0,0.0,0.0,202128,202328,0,50.208904,0.0
"""ZZ90239""",202624,0.0,0.0,0.0,202128,202328,0,50.208904,0.0


In [ ]:

import matplotlib.pyplot as plt
from tqdm import tqdm

parts = df_norm.select('oper_part_no').unique().to_series().to_list()

show_parts = parts[1:2]
plt.figure(figsize = (32, 12))

for p in tqdm(show_parts):
    print(p)
    gi = df_norm.filter(pl.col('oper_part_no') == p).sort('yyyyww')
    if gi.is_empty(): continue
    gpd = gi.select(['yyyyww', 'demand_z'])
    x = gpd['yyyyww'].to_numpy()
    y = gpd['demand_z'].to_numpy()

    plt.plot(x, y, linewidth = 1.8, alpha = 0.35)

plt.title(f"Normalized demand trend (robust min-max)  |  parts shown = {len(parts)}")
plt.xlabel("yyyyww")
plt.ylabel("demand_mm_robust (0~1)")
plt.ylim(-0.05, 1.05)
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()